# Stock Price Prediction

### Load Dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import math 
import pandas_datareader as web
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

In [ ]:
df = web.DataReader('AAPL', data_source='yahoo', start='2016-03-01', end='2021-01-01')
df.head()

In [ ]:
df.shape

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(df["Close"])
plt.title('Close Price Data')
plt.xlabel('Time')
plt.ylabel('Close Price USD ($)')
plt.show()

### Preprocess

In [ ]:
close_data = df.filter(["Close"])
dataset = close_data.values
train_data_len = math.ceil(len(dataset) * 0.8)

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

In [ ]:
# Train data
train_data = scaled_data[:train_data_len, :]
x_train, y_train = [], []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
    
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

# Test data
test_data = scaled_data[train_data_len-60:, :]
x_test = []
y_test = dataset[train_data_len:, :]

for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])
    
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

### Training

In [ ]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dropout(0.2))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')

model.fit(x_train, y_train, epochs=10, batch_size=16)

# model.save('spp_model.hdf5')

### Evaluation

In [ ]:
pred = model.predict(x_test)
pred = scaler.inverse_transform(pred)

In [ ]:
rms=np.sqrt(np.mean(np.power((pred-y_test), 2)))
print(rms)

In [ ]:
train = close_data[:train_data_len]
valid = close_data[train_data_len:]
valid['Predictions'] = pred
plt.figure(figsize=(16,8))
plt.plot(train["Close"])
plt.plot(valid[['Close', "Predictions"]])
plt.title('Close Price Predictions')
plt.xlabel('Time')
plt.ylabel('Close Price USD ($)')
plt.legend(['Train', 'Val', 'Predictions'])
plt.show()

### Predictions

In [ ]:
df_pred = web.DataReader('AAPL', data_source='yahoo', start='2016-03-01', end='2021-05-01')
df_pred.head()

In [ ]:
last_60_days = df_pred.filter(["Close"])[-60:]
last_60_days_data = last_60_days.values
last_60_days = scaler.transform(last_60_days_data)
x_last_60_test = [last_60_days]
x_last_60_test = np.array(x_last_60_test)
x_last_60_test = np.reshape(x_last_60_test, (x_last_60_test.shape[0], x_last_60_test.shape[1], 1))

pred_last_60 = model.predict(x_last_60_test)
pred_last_60 = scaler.inverse_transform(pred_last_60)

In [ ]:
# Compare 
last_date = df_pred[-1:].index[0]
df_real = web.DataReader('AAPL', data_source='yahoo', start=last_date, end=last_date)
print(f"prediction: {pred_last_60[0][0]}, actual: {df_real['Close'][0]}")

In [ ]:
'''
Inspiration
1. https://youtu.be/QIUxPv5PJOY
'''